In [1]:
import pandas as pd

In [2]:
# titanic data set #
df = pd.read_csv("http://bit.ly/kaggletrain")

In [3]:
df.shape

(891, 12)

In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
# null values col #
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
len(df["Embarked"].notna())

891

In [12]:
# exclude entries where Embarked is null #
df = df.loc[df["Embarked"].notna(), ["Survived","Pclass","Sex","Embarked"]]

In [14]:
df.shape

(889, 4)

In [15]:
# Check null values #
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Embarked    0
dtype: int64

In [16]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


**Cross validating a model that predicts Survived using only Pclass**

In [18]:
X = df.loc[:, ["Pclass"]]
y = df["Survived"]

In [24]:
X.shape

(889, 1)

In [25]:
y.shape

(889,)

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
logreg = LogisticRegression(solver="lbfgs")

In [30]:
from sklearn.model_selection import cross_val_score

In [31]:
cross_val_score(logreg, X, y, cv=5, scoring="accuracy").mean()

0.6783406335301212

In [34]:
# null accuracy for predicting the most frequent class #
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

**Add more independant variables**

In [35]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [36]:
# Encoding categorical features #
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

In [37]:
# Sex col Example #
ohe.fit_transform(df[["Sex"]])

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [38]:
# View what each array stands for #
ohe.categories_

[array(['female', 'male'], dtype=object)]

In [39]:
# Example of Embarked col #
ohe.fit_transform(df[["Embarked"]])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [40]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

**Using Pipeline with one hot encoding**

In [41]:
# Features #
X = df.drop("Survived", axis="columns")

In [42]:
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


column transformer is used when you have columns in your data frame that need different Preprocessing

In [43]:
# column transformer #
from sklearn.compose import make_column_transformer

In [44]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ["Sex", "Embarked"]),
    remainder="passthrough"
)

In [45]:
column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [51]:
# Put all these steps into a Pipeline #
from sklearn.pipeline import make_pipeline

In [52]:
# Order of application. First colum transform then apply logistic regression #
pipe = make_pipeline(column_trans, logreg)

In [53]:
# Passing entire pipeline into cross validation score #
cross_val_score(pipe, X, y, cv=5, scoring="accuracy").mean()

0.7727924839713071

**Predictions on new data**

In [54]:
X_new = X.sample(5, random_state=99)
X_new

,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


In [55]:
pipe.fit(X, y)

Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('onehotencoder', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [56]:
# The pipeline is automaticaly doing the dummy encoding of the new data #
pipe.predict(X_new)

array([1, 0, 1, 1, 0], dtype=int64)